In [1]:
import numpy as np
import random

In [2]:
# Generate a random gridworld
num_rows = random.randint(3, 6)
num_cols = random.randint(3, 6)
gridworld = np.random.choice([0, 1], size=(num_rows, num_cols), p=[0.8, 0.2])

# Randomly place the goal cell (-1)
goal_row = random.randint(0, num_rows - 1)
goal_col = random.randint(0, num_cols - 1)
gridworld[goal_row, goal_col] = -1
print(gridworld)

# Set the place where the agent starts
start_row = random.randint(0, num_rows - 1)
start_col = random.randint(0, num_cols - 1)
current_state = (start_row, start_col)
print(current_state)

[[ 0 -1  0  0  0  0]
 [ 0  0  1  0  0  1]
 [ 0  1  1  0  0  0]]
(1, 0)


In [3]:
# Define constants
NUM_EPISODES = 1000
MAX_STEPS = 100
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration rate

In [4]:
# Define the state and action spaces
num_rows = len(gridworld)
num_cols = len(gridworld[0])
state_space = range(num_rows * num_cols)
action_space = ['north', 'south', 'east', 'west']

In [5]:
# Initialize the Q-table with zeros
q_table = np.zeros((num_rows * num_cols, len(action_space)))

In [6]:
# Convert a (row, col) state to an index
def state_to_index(state):
    row, col = state
    return row * num_cols + col


In [7]:
# Convert an index to a (row, col) state
def index_to_state(index):
    row = index // num_cols
    col = index % num_cols
    return row, col

In [8]:
# Perform Q-learning iterations
for episode in range(NUM_EPISODES):
    # Reset the environment
    total_reward = 0
    
    for step in range(MAX_STEPS):
        # Choose an action using epsilon-greedy policy
        if random.uniform(0, 1) < EPSILON:
            action = random.choice(action_space)
        else:
            current_index = state_to_index(current_state)
            q_values = q_table[current_index]
            action = action_space[np.argmax(q_values)]
        
        # Execute the chosen action and observe the next state
        if action == 'north':
            next_state = (current_state[0] - 1, current_state[1])
        elif action == 'south':
            next_state = (current_state[0] + 1, current_state[1])
        elif action == 'east':
            next_state = (current_state[0], current_state[1] + 1)
        elif action == 'west':
            next_state = (current_state[0], current_state[1] - 1)
        
        # Handle boundary conditions
        if next_state[0] < 0 or next_state[0] >= num_rows or next_state[1] < 0 or next_state[1] >= num_cols:
            next_state = current_state
        
        # Get the reward for the current state-action pair
        current_index = state_to_index(current_state)
        next_index = state_to_index(next_state)
        reward = gridworld[current_state[0]][current_state[1]]
        
        # Update the Q-value for the current state-action pair
        q_table[current_index, action_space.index(action)] = (1 - ALPHA) * q_table[current_index, action_space.index(action)] + \
                                                             ALPHA * (reward + GAMMA * np.max(q_table[next_index]))
        
        total_reward += reward
        current_state = next_state
        
        # Check if reached the goal state
        if reward == -1:
            break
    
    # Print the total reward for the episode
    print(f"Episode {episode+1} - Total Reward: {total_reward}")

Episode 1 - Total Reward: -1
Episode 2 - Total Reward: -1
Episode 3 - Total Reward: -1
Episode 4 - Total Reward: 23
Episode 5 - Total Reward: -1
Episode 6 - Total Reward: -1
Episode 7 - Total Reward: -1
Episode 8 - Total Reward: 5
Episode 9 - Total Reward: 0
Episode 10 - Total Reward: 0
Episode 11 - Total Reward: 9
Episode 12 - Total Reward: 9
Episode 13 - Total Reward: 22
Episode 14 - Total Reward: 9
Episode 15 - Total Reward: 36
Episode 16 - Total Reward: 42
Episode 17 - Total Reward: -1
Episode 18 - Total Reward: 9
Episode 19 - Total Reward: 2
Episode 20 - Total Reward: 45
Episode 21 - Total Reward: 29
Episode 22 - Total Reward: 14
Episode 23 - Total Reward: 11
Episode 24 - Total Reward: 2
Episode 25 - Total Reward: 1
Episode 26 - Total Reward: 49
Episode 27 - Total Reward: 28
Episode 28 - Total Reward: 45
Episode 29 - Total Reward: 42
Episode 30 - Total Reward: -1
Episode 31 - Total Reward: 15
Episode 32 - Total Reward: 35
Episode 33 - Total Reward: 48
Episode 34 - Total Reward: 31

In [9]:
# Extract the optimal policy from the learned Q-table
optimal_policy = []
for state in state_space:
    row, col = index_to_state(state)
    if gridworld[row][col] != 1:
        q_values = q_table[state]
        best_action = action_space[np.argmax(q_values)]
        optimal_policy.append(best_action)
    else:
        optimal_policy.append('-')  # Mark walls as invalid actions

In [10]:
# Print the optimal policy
print("\nOptimal Policy:")
for row in range(num_rows):
    for col in range(num_cols):
        print(optimal_policy[state_to_index((row, col))], end='\t')
    print()


Optimal Policy:
east	east	south	west	west	west	
north	south	-	west	north	-	
north	-	-	north	north	north	
